In [5]:
import pandas as pd

# Chargement du fichier de température (Berkeley Earth .txt)
# On ignore les lignes de commentaire qui commencent par '%'
berkeley_path = "/Users/jeniferawounou/Desktop/france-(europe)-TAVG-Trend-bon.txt"
df_temp = pd.read_csv(berkeley_path, comment='%', delim_whitespace=True, header=None)

# Renommer les colonnes pour plus de clarté
df_temp.columns = [
    "Year", "Month", "Monthly_Anomaly", "Monthly_Unc",
    "Annual_Anomaly", "Annual_Unc", "FiveYear_Anomaly", "FiveYear_Unc",
    "TenYear_Anomaly", "TenYear_Unc", "TwentyYear_Anomaly", "TwentyYear_Unc"
]

# Création d'une colonne 'Date' pour l'analyse temporelle
df_temp['Date'] = pd.to_datetime(dict(year=df_temp.Year, month=df_temp.Month, day=1))

# Chargement du fichier EM-DAT (.xlsx)
emdat_path = "/Users/jeniferawounou/Desktop/public_emdat_custom_request_2025-04-01_955b1370-4c28-4320-97f2-9b207c3e884d.xlsx"

# Lecture du fichier Excel (après avoir installé openpyxl)
df_disasters = pd.read_excel("/Users/jeniferawounou/Desktop/public_emdat_custom_request_2025-04-01_955b1370-4c28-4320-97f2-9b207c3e884d.xlsx", engine="openpyxl")


# Afficher les premières lignes de chaque dataset pour vérification
df_temp.head(), df_disasters.head()


/var/folders/5g/dxyz07h551b0md8v8dx0w1w80000gn/T/ipykernel_35222/3752440225.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_temp = pd.read_csv(berkeley_path, comment='%', delim_whitespace=True, header=None)


(   Year  Month  Monthly_Anomaly  Monthly_Unc  Annual_Anomaly  Annual_Unc  \
 0  1750      1           -1.830        1.837             NaN         NaN   
 1  1750      2            0.220        2.099          -0.391       0.798   
 2  1750      3            1.858        2.539          -0.340       0.820   
 3  1750      4            0.266        1.609          -0.315       0.822   
 4  1750      5           -0.785        1.603          -0.462       0.822   
 
    FiveYear_Anomaly  FiveYear_Unc  TenYear_Anomaly  TenYear_Unc  \
 0               NaN           NaN              NaN          NaN   
 1               NaN           NaN              NaN          NaN   
 2               NaN           NaN              NaN          NaN   
 3               NaN           NaN              NaN          NaN   
 4               NaN           NaN              NaN          NaN   
 
    TwentyYear_Anomaly  TwentyYear_Unc       Date  
 0                 NaN             NaN 1750-01-01  
 1                 NaN

In [3]:
!pip install openpyxl



  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [7]:
# Afficher les noms de colonnes du fichier EM-DAT pour repérer le vrai nom de "Year"
df_disasters.columns


Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update'],
      dtype='object')

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [19]:
# Recalcul de df_france suite à une perte de contexte
# Normalisation du nom de pays
df_disasters['Country'] = df_disasters['Country'].str.strip().str.lower()
df_france = df_disasters[df_disasters['Country'] == 'france']

# Refaire l'agrégation annuelle
df_disasters_yearly = df_france.groupby('Start Year').agg({
    'Disaster Type': 'count',
    'Total Deaths': 'sum',
    "Total Damage ('000 US$)": 'sum'
}).reset_index()

# Renommer les colonnes pour clarté
df_disasters_yearly.columns = ['Year', 'Disaster_Count', 'Total_Deaths', 'Total_Damage']

# Préparer les températures moyennes annuelles depuis 1900
df_temp_filtered = df_temp[df_temp['Year'] >= 1900]
df_temp_yearly = df_temp_filtered.groupby('Year').agg({
    'Monthly_Anomaly': 'mean'
}).reset_index()

# Fusion des deux datasets sur l'année
df_merged = pd.merge(df_temp_yearly, df_disasters_yearly, on='Year', how='inner')

print(df_merged.head(20))
import os

# Vérifie le chemin courant (racine du projet)
print("Chemin actuel :", os.getcwd())

# Sauvegarde du fichier dans ce dossier
df_merged.to_csv("donnees_fusionnees_france.csv", index=False)
print("✅ Fichier sauvegardé dans la racine du projet !")



    Year  Monthly_Anomaly  Disaster_Count  Total_Deaths  Total_Damage
0   1949         1.020917               1          80.0           0.0
1   1982         0.891417               1           2.0           0.0
2   1983         0.636667               1          12.0           0.0
3   1984        -0.053083               2           0.0           0.0
4   1985        -0.340500               1           6.0           0.0
5   1986        -0.078833               1           5.0           0.0
6   1989         1.227917               1           0.0     1600000.0
7   1997         1.290000               2           0.0       10000.0
8   1999         1.115833               1           0.0           0.0
9   2003         1.578083               1           5.0           0.0
10  2005         0.733667               1           2.0       10000.0
11  2009         1.123667               1           0.0           0.0
12  2017         1.337083               1           0.0           0.0
Chemin actuel : /Use